In [21]:
# Problems 8-10 on https://work.caltech.edu/homework/hw2.pdf

import numpy as np

def runs(n, run_count=1000):
    total_e_in = 0

    for i in range(0, run_count):
        e_in = run(n)
        total_e_in += e_in

    avg_e_in = total_e_in / run_count

    return avg_e_in

def runs_transformed(n, run_count = 1000):
    total_weights = np.zeros(6)
    total_e_out = 0

    for i in range(0, run_count):
        weights, e_out = run_transformed(n)
        total_weights += weights
        total_e_out += e_out

    avg_weights = total_weights / run_count
    avg_e_out = total_e_out / run_count

    return avg_weights, avg_e_out

# Note: I forgot to introduce noise here per the problem, but still go the right answer.
def run(n):
    rng = np.random.default_rng()
    xs = rng.uniform(-1,1,(n,2))
    # Our target function here
    line_heights = np.sum(np.square(xs), 1) - .6
    ys = np.sign(line_heights)
    xs = np.hstack((np.ones((n,1)), xs)) # Set x_0 = 1 for all xs

    weights = pseudo_inverse(xs) @ ys

    predicted_ys = np.sign(xs @ weights)
    misclassified_points = np.flatnonzero(predicted_ys != ys)

    # E_in = the fraction of in-sample points which got classified incorrectly
    e_in = len(misclassified_points) / n

    return e_in

def run_transformed(n):
    rng = np.random.default_rng()
    xs = rng.uniform(-1,1,(n,2))
    line_heights = np.sum(np.square(xs), 1) - .6
    ys = np.sign(line_heights)
    
    # We are introducing 10% noise
    mask = rng.choice(n, size=int(n/10), replace=False)
    ys[mask] *= -1

    xs = np.hstack((xs, 
        (xs[:, 0] * xs[:, 1]).reshape(n,1),
        (np.square(xs[:, 0])).reshape(n,1),
        (np.square(xs[:, 1])).reshape(n,1)))
    xs = np.hstack((np.ones((n,1)), xs)) # Set x_0 = 1 for all xs
    
    weights = pseudo_inverse(xs) @ ys

    # Generate 1000 random points and determine how many disagree
    xs = rng.uniform(-1,1,(1000,2))
    line_heights = np.sum(np.square(xs), 1) - .6
    ys = np.sign(line_heights)

    # We are introducing 10% noise
    mask = rng.choice(1000, size=int(1000/10), replace=False)
    ys[mask] *= -1

    xs = np.hstack((xs, 
        (xs[:, 0] * xs[:, 1]).reshape(1000,1),
        (np.square(xs[:, 0])).reshape(1000,1),
        (np.square(xs[:, 1])).reshape(1000,1)))
    xs = np.hstack((np.ones((1000,1)), xs)) # Set x_0 = 1 for all xs
    predicted_ys = np.sign(xs @ weights)
    misclassified_count = len(np.flatnonzero(predicted_ys != ys))

    e_out = misclassified_count / 1000

    return weights, e_out

def pseudo_inverse(xs):
    return np.linalg.inv(xs.T @ xs) @ xs.T

# print(runs(1000))
avg_weights, avg_e_out = runs_transformed(10000)
print(avg_e_out)
print([float(el) for el in avg_weights])


0.12313699999999987
[-0.9908744758956698, 0.00035599386982252366, 0.0003215973472249168, 0.00023488518848405616, 1.5555419436096132, 1.5564105717999668]
